Demo of diffuse and specular reflectance on a sphere.

In [0]:
###########
# Imports #
###########
import numpy as np
import math as m
import tensorflow as tf
import matplotlib.pyplot as plt
from colabtools import interactive_forms

from colabtools import adhoc_import
with adhoc_import.Google3():
  from tensorflow_graphics.reflectance import lambertian
  from tensorflow_graphics.reflectance import phong
  from tensorflow_graphics.camera import orthographic
  from tensorflow_graphics.geometry import grid
  from tensorflow_graphics.geometry import vector

############################################################################################
# Helper function allowing to estimate sphere normal and depth for each pixel in the image #
############################################################################################
def compute_intersection_normal_sphere(image_width, image_height, sphere_radius,
                                       sphere_center, type):
  pixel_grid_start = np.array((0.5, 0.5), dtype=type)
  pixel_grid_end = np.array((image_width - 0.5, image_height - 0.5), dtype=type)
  pixel_nb = np.array((image_width, image_height))
  pixels = grid.generate(pixel_grid_start, pixel_grid_end, pixel_nb)

  pixel_ray = tf.math.l2_normalize(orthographic.ray(pixels), axis=-1)
  zero_depth = np.zeros([image_width, image_height, 1])
  pixels_3d = orthographic.unproject(pixels, zero_depth)
  vector_sphere_center_to_pixel = sphere_center - pixels_3d
  distance_sphere_center_to_pixel = tf.norm(
      vector_sphere_center_to_pixel, axis=-1, keepdims=True)
  distance_sphere_centre_on_ray = vector.dot(vector_sphere_center_to_pixel,
                                             pixel_ray)
  distance_sphere_centre_to_ray = tf.sqrt(
      tf.pow(distance_sphere_center_to_pixel, 2) -
      tf.pow(distance_sphere_centre_on_ray, 2))

  p = tf.sqrt(
      tf.pow(sphere_radius, 2) - tf.pow(distance_sphere_centre_to_ray, 2))
  distance_ray_sphere_surface = distance_sphere_centre_on_ray - p

  outside = tf.zeros_like(p, dtype=bool)
  inside = tf.ones_like(p, dtype=bool)
  mask = tf.where(p <= sphere_radius, inside, outside)
  mask = tf.tile(mask, (1, 1, 3))

  zeros = tf.zeros_like(pixels_3d)
  intersection_3d = tf.where(
      mask, pixel_ray * distance_ray_sphere_surface + pixels_3d, zeros)
  surface_normal = tf.where(
      mask, tf.math.l2_normalize(intersection_3d - sphere_center, axis=-1),
      zeros)
  return intersection_3d, surface_normal

In [0]:
###############
# UI controls #
###############
#@title Controls { vertical-output: false, run: "auto" }
light_x_position = -0.45  #@param { type: "slider", min: -1, max: 1 , step: 0.05 }
albedo_red = 0.7  #@param { type: "slider", min: 0.0, max: 1.0 , step: 0.1 }
albedo_green = 1  #@param { type: "slider", min: 0.0, max: 1.0 , step: 0.1 }
albedo_blue = 1  #@param { type: "slider", min: 0.0, max: 1.0 , step: 0.1 }
light_red = 1  #@param { type: "slider", min: 0.0, max: 1.0 , step: 0.1 }
light_green = 1  #@param { type: "slider", min: 0.0, max: 1.0 , step: 0.1 }
light_blue = 1  #@param { type: "slider", min: 0.0, max: 1.0 , step: 0.1 }
specular_percentage = 0.46  #@param { type: "slider", min: 0, max: 1 , step: 0.01 }
shininess = 9  #@param { type: "slider", min: 0, max: 10, step: 1 }
diffuse_percentage = 1.0 - specular_percentage
type = np.float64
albedo = np.array((albedo_red, albedo_green, albedo_blue), dtype=type)

#####################################
# Setup the image, sphere and light #
#####################################
# Image dimensions
image_width = 400
image_height = 300

# Sphere center and radius
sphere_radius = np.array(100, dtype=type)
sphere_center = np.array((image_width / 2.0, image_height / 2.0, 300.0),
                         dtype=type)

# Set the light along the image plane
light_position = np.array(
    (image_width / 2 + light_x_position * image_width, image_height / 2.0, 0.0),
    dtype=type)
vector_light_to_sphere_center = light_position - sphere_center
light_intensity_scale = vector.dot(
    vector_light_to_sphere_center, vector_light_to_sphere_center, axis=-1) * 4 * m.pi
light_intensity = np.array(
    (light_red, light_green, light_blue)) * light_intensity_scale

################################################################################################
# For each pixel in the image, estimate the corresponding surface point and associated normal. #
################################################################################################
intersection_3d, surface_normal = compute_intersection_normal_sphere(
    image_width, image_height, sphere_radius, sphere_center, type)

#######################################
# Reflectance and radiance estimation #
#######################################
incoming_light_direction = tf.math.l2_normalize(
    intersection_3d - light_position, axis=-1)
# Lambertian BRDF
brdf_lambertian = lambertian.brdf(albedo)
# Phong BRDF
outgoing_ray = np.array((0.0, 0.0, -1.0), dtype=type)
brdf_phong = phong.brdf(incoming_light_direction, outgoing_ray, surface_normal,
                        np.array((shininess,), dtype=type), albedo)
# Composite BRDF
brdf_composite = diffuse_percentage * brdf_lambertian + specular_percentage * brdf_phong
# Irradiance
cosine_term = vector.dot(surface_normal, -incoming_light_direction)
cosine_term = tf.math.maximum(tf.zeros_like(cosine_term), cosine_term)
vector_light_to_surface = intersection_3d - light_position
light_to_surface_distance_squared = vector.dot(
    vector_light_to_surface, vector_light_to_surface, axis=-1)
irradiance = light_intensity / (4 * m.pi * light_to_surface_distance_squared) * cosine_term
# Rendering equation
zeros = tf.zeros(intersection_3d.shape)
radiance = brdf_composite * irradiance
radiance_lambertian = brdf_lambertian * irradiance
radiance_phong = brdf_phong * irradiance

with tf.name_scope("initialize_variables"):
  init = tf.initialize_all_variables()

session = tf.Session()
session.run(init)
radiance_, radiance_lambertian_, radiance_phong_ = session.run(
    [radiance, radiance_lambertian, radiance_phong])

###############################
# Display the rendered sphere #
###############################
# Saturates radiances at 1 for rendering purposes.
radiance_ = np.minimum(radiance_, 1.0)
radiance_lambertian_ = np.minimum(radiance_lambertian_, 1.0)
radiance_phong_ = np.minimum(radiance_phong_, 1.0)
# Gammma correction
radiance_ = np.power(radiance_, 1.0 / 2.2)
radiance_lambertian_ = np.power(radiance_lambertian_, 1.0 / 2.2)
radiance_phong_ = np.power(radiance_phong_, 1.0 / 2.2)

plt.figure(figsize=(20, 20))

# Diffuse + specular
radiance_ = np.transpose(radiance_, (1, 0, 2))
ax = plt.subplot("131")
ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)
ax.grid(False)
ax.set_title("Blend diffuse and specular")
_ = ax.imshow(radiance_)

# Diffuse
radiance_lambertian_ = np.transpose(radiance_lambertian_, (1, 0, 2))
ax = plt.subplot("132")
ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)
ax.grid(False)
ax.set_title("Lambertian")
_ = ax.imshow(radiance_lambertian_)

# Specular
radiance_phong_ = np.transpose(radiance_phong_, (1, 0, 2))
ax = plt.subplot("133")
ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)
ax.grid(False)
ax.set_title("Phong")
_ = ax.imshow(radiance_phong_)